In [53]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# import from custom package
from auxFuns.load_data import *

In [54]:
make_it_categorical

<function auxFuns.load_data.make_it_categorical(df, is_phase1=True)>

In [71]:
# os.chdir('..')

# 1. Loading the data and summary function

In [70]:
datasets_path = os.getcwd() + '/datasets/raw'

In [72]:
rsv_predictors_phase1_df = pd.read_csv(datasets_path + '/rsv_predictors_df_phase1_v3.csv')
rsv_predictors_phase1_df = make_it_categorical(rsv_predictors_phase1_df)

In [73]:
summary_function_rsv(rsv_predictors_phase1_df)

                CCI          sine        cosine    n_symptoms  n_encounters  \
count  86659.000000  86659.000000  8.665900e+04  86659.000000  86659.000000   
mean       2.360297      0.305377  3.863281e-01      2.145963      4.697642   
std        2.432005      0.639752  5.901029e-01      1.702432      6.967793   
min        0.000000     -1.000000 -1.000000e+00      0.000000      1.000000   
25%        0.000000     -0.239316 -1.608123e-16      1.000000      1.000000   
50%        2.000000      0.464723  5.680647e-01      2.000000      2.000000   
75%        4.000000      0.885456  8.854560e-01      3.000000      5.000000   
max       14.000000      1.000000  1.000000e+00     13.000000     79.000000   

       prev_positive_rsv  tendency_to_positivity  previous_test_daydiff  \
count       86659.000000            86659.000000           8.665900e+04   
mean            0.061159                0.030626           3.788330e+05   
std             0.394196                0.127717           4.77

# 2. Plots of positivity and evaluation of correlations

# 3. Evaluate positivity within a short period of time
- Tests in the same day and same patient
- Tests in the same week (and same patient)

In [74]:
rsv_predictors_phase1_df['index_date'] = pd.to_datetime(rsv_predictors_phase1_df['index_date']).dt.date

In [103]:
# I want to see (same Day)
# a) how many tests are taken at the same day and same patient 
# b) how many have i) positive-positive, ii) negative-negative, iii) negative-positive

# Take only those records having more than one test that day
more_one_rsv_test_df = (
    rsv_predictors_phase1_df.loc[rsv_predictors_phase1_df['n_tests_that_day'] == 2, 
                                 ['patient_id', 'index_date', 'RSV_test_result']]
                                 )

more_one_rsv_test_df.shape # 1202 records, i.e. 601 test records the same day

(more_one_rsv_test_df['RSV_test_result'] == 'Positive').sum() # 601
(more_one_rsv_test_df['RSV_test_result'] == 'Negative').sum() # 601

# It seems that, every time a test is repeated, there exists one negative and positive test

601

In [106]:
more_one_rsv_test_df.sort_values(by = 'patient_id')

,patient_id,index_date,RSV_test_result
85528,006f8d2dd2a3b7e3d98761402281bf358c1e255e,2016-02-22,Positive
85527,006f8d2dd2a3b7e3d98761402281bf358c1e255e,2016-02-22,Negative
50844,007ddd177a2d6eb4dff4eec31fb9262e8e3b6774,2020-02-25,Negative
50840,007ddd177a2d6eb4dff4eec31fb9262e8e3b6774,2020-02-25,Positive
14235,00d27ca6dc2904cc64efe2e8031216fd0cf59122,2017-05-14,Positive
...,...,...,...
72246,feb29fbbd8877a4a4e0feaa2f7bb3f893ce28c5d,2019-03-18,Positive
71595,ff636c272046e8908e6cb9290423701bab9bd877,2015-01-16,Negative
71594,ff636c272046e8908e6cb9290423701bab9bd877,2015-01-16,Positive
60486,ffe576cf11eb5b4cc2b406cdde4f541d85a17dfa,2013-02-11,Positive
